In [ ]:
%pip install -U langchain-google-genai

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from tqdm.auto import tqdm
from time import sleep

# from langchain import Langchain
from langchain_google_genai import ChatGoogleGenerativeAI


load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# lc = Langchain(api_key=GEMINI_API_KEY)
# status = lc.check_connection()
# print(f"Gemini connection status: {status}")

llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=GEMINI_API_KEY,
)
# messages = [('system', "You are a helpful assistant. "),
            # ('human', "Who was Carl Sagan?")]
# resp = llm.invoke(messages)
# print(resp.content)

def generate_example(target_word, vocab_set):
    augmented_vocab = vocab_set | {target_word}
    system_prompt = f"You are a helpful language tutor. Your student is only familiar with the following words, so please mainly use words from the following list: {augmented_vocab}"
    user_prompt = f"Please write a sentence using the word \"{target_word}\", with enough context clues that someone can understand the meaning of the word. However, don't simply define the word. Respond with only the sentence, nothing else, no explanations."

    # print(prompt)
    messages = [('system', system_prompt),
                ('human', user_prompt)]
    resp = llm.invoke(messages)
    return resp

def main():

    common_vocab = pd.read_csv("../data/vocab_2000.csv")
    common_500 = {w.lower() for w in common_vocab['word'][:500]}

    # scuffed but works in a pinch
    target_words = pd.read_csv("../data/output_0.csv")['vocab_word'][:34]

    # resps = []
    data = []
    for target_word in tqdm(target_words):

        resp = generate_example(target_word, common_500)
        data.append({'vocab_word': target_word, 
        'baseline_prompt': resp.content,
        'boost_coefficient': 'gemini'}
        )
        sleep(4) # rate limit: 15 requests per minute

    result = pd.DataFrame(data)
    result.to_csv("../data/output_gemini.csv")

main()
